In [ ]:
%matplotlib widget
from matplotlib import pyplot as plt
import numpy as np
from ipywidgets import interact

# SIR epidemics model interactive simulation  
SIR models are based on the assumption that each individual is either Susceptible, Infectious, or Removed (removed are NOT considered susceptible anymore, hence they cannot spread the disease).  
The simplest SIR model also assumes a constant population N = S(t) + I(t) + R(t) at any given time t.  
One of the key ideas behind the SIR model is that people unidirectionally flow between categories in the order S -> I -> R  
  
The following three differential equations describe the change in population count in each of the categories:  
![equation](https://latex.codecogs.com/png.latex?%5Cbg_white%20%5Cboxed%7B%20%5Cbegin%7Balign*%7D%20%26%5Cfrac%7B%5Cmathrm%7Bd%7D%20S%7D%7B%5Cmathrm%7Bd%7D%20t%7D%20%3D%20-%5Cfrac%7B%5Cbeta%20IS%7D%7BN%7D%20%5C%5C%5B15pt%5D%20%26%5Cfrac%7B%5Cmathrm%7Bd%7D%20I%7D%7B%5Cmathrm%7Bd%7D%20t%7D%20%3D%20%5Cfrac%7B%5Cbeta%20IS%7D%7BN%7D%20-%20%5Cgamma%20I%20%5C%5C%5B15pt%5D%20%26%5Cfrac%7B%5Cmathrm%7Bd%7D%20R%7D%7B%5Cmathrm%7Bd%7D%20t%7D%20%3D%20%5Cgamma%20I%20%5Cend%7Balign*%7D%20%7D)

The number of susceptible people that become infectious as time goes by depends on the population count of both categories and the contagion coefficient (higher means the disease spreads faster)  
By contrast, as the number of susceptible people decreases, the number of infectious increases by the same amount. Some people are going to recover (or sadly die) from the disease, so the number of infectious also decreases depending on the removal coefficient and the current infectious count.
Lastly,  the number of removed people increases by the same amount described above.

In [ ]:
# SIR constants
N = 10000       # total population
R0 = 0          # initial removed people

# beta = 0.4          # contagion coefficient exampla
# gamma = 0.04        # removal coefficient example

The simulation uses the [Euler method](https://en.wikipedia.org/wiki/Euler_method) to calculate curves.  
It consists in approximating the function taking small steps on the tangents instead of perfectly following the curve.  
The resulting approximation usually is quite good, depending on the chosen step size

In [ ]:
# Eulerian approximation constants
h = 0.5
end = 150

t = np.arange(0, end, h)
s = np.zeros(int(end/h))
i = np.zeros(int(end/h))
r = np.zeros(int(end/h))

In [ ]:
# s, i, r are allocated just once and modified in simulate_sir to avoid unnecessary memory allocations
def simulate_sir(I0, beta, gamma, s, i, r):
    '''
        I0 = initial number of infectious
        beta = contagion coefficient
        gamma = removal coefficient
    '''
    s[0] = N-I0-R0     # using N = S + I + R
    i[0] = I0
    r[0] = R0
    
    for j in range(1, len(t)):
        # calculate the update of the three differencial equations 
        ds = -(beta * i[j-1] * s[j-1])/N
        di = (beta * i[j-1] * s[j-1])/N - gamma*i[j-1]
        dr = gamma * i[j-1]
        # update values with step h
        s[j] = s[j-1] + h*ds
        i[j] = i[j-1] + h*di
        r[j] = r[j-1] + h*dr

In [ ]:
fig = plt.figure(figsize=(9, 5))
ax = plt.axes(xlim=(0, 150), ylim=(0, 10000))

# plt.plot(x, y) returns a list of Line2D objects with only one line.
# we want three of them to update our plot setting sir data without recreating the plot multiple times
lines = [plt.plot([], [])[0] for _ in range(3)]

@interact
def simulate_changing_params(I_zero=(1, 100, 1), beta=(0.04, 1, 0.01), gamma=(0.004, 0.4, 0.001)):
    simulate_sir(I_zero, beta, gamma, s, i, r)
    lines[0].set_data(t, s)
    lines[1].set_data(t, i)
    lines[2].set_data(t, r)